# Benchmark GPU vs CPU
---

This notebook can be used to benchmark performance using CPU, a single GPU or many GPUs.

In [1]:
def wrapper():
    import tensorflow as tf
    
    # Wrapper for keras_applications, you can import any model you want to try (like ResNet50)
    from tensorflow.keras.applications import VGG19

    import numpy as np
    
    from hops import tensorboard
    
    # Utility module for getting number of GPUs accessible by the container (Spark Executor)
    from hops import devices

    batch_size = 32 # Number of samples to process on each GPU every iteration
    
    # Image dimensions
    height = 224
    width = 224
    channels = 3
    num_classes = 1000
    
    num_iterations = 5000 # Number of iterations, increase to run longer
    
    model = VGG19(weights=None, input_shape=(height, width, channels), classes=num_classes)
        
    optimizer = tf.train.RMSPropOptimizer(0.2)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    
    # Read synthetic data (can be replaced with real data)
    def input_fn():
      data = np.random.random((batch_size, height, width, channels))
      labels = np.random.random((batch_size, num_classes))
      x = tf.cast(data, tf.float32)
      dataset = tf.data.Dataset.from_tensor_slices((data, labels))
      dataset = dataset.repeat(num_iterations)
      dataset = dataset.batch(batch_size)
      return dataset    
    
    tf.keras.backend.set_learning_phase(True)

    # Define DistributionStrategies and convert the Keras Model to an
    # Estimator that utilizes these DistributionStrateges.
    # Evaluator is a single worker, so using MirroredStrategy.
    # Training is automatically distributed on all available GPUs when using MirroredStrategy
    run_config = tf.estimator.RunConfig(
            train_distribute=tf.contrib.distribute.MirroredStrategy())
    keras_estimator = tf.keras.estimator.model_to_estimator(keras_model=model, 
               config=run_config, model_dir=tensorboard.logdir())

    # Train and evaluate the model. Evaluation will be skipped if there is not an
    # "evaluator" job in the cluster.
    tf.estimator.train_and_evaluate(keras_estimator, train_spec=tf.estimator.TrainSpec(input_fn=input_fn),
        eval_spec=tf.estimator.EvalSpec(input_fn=input_fn))
        

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1102,application_1549714785003_1348,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [ ]:
from hops import experiment
experiment.launch(wrapper, local_logdir=True)